In [ ]:
from importlib import reload
from scipy.optimize import curve_fit
import os
import data_processing as dp
import numpy as np
import matplotlib.pyplot as plt
dp = reload(dp)

In [ ]:
def dir_filter(directory):
    dir_name = directory.name
    name_index = dir_name.split("_")
    return name_index[0] == "explore"

root = os.getcwd()
output_path = os.path.join(root, "..", "output")
dirs = list(os.scandir(output_path))

# observables to investigate
obs_names = ["dual_graph_shell", "dual_tree_shell", "edge_degree", "edge_detour", "edge_freq", "edge_middle", "triangle_detour", "triangle_freq", "triangle_middle", "vertex_count", "vertex_degree", "vertex_tree_shell", "volume"]
#obs_names = ["dual_dist_prof", "edge_degree", "edge_detour", "edge_middle", "triangle_detour", "triangle_middle", "vertex_count", "vertex_degree", "volume"]

# build a container
all_data = np.empty((len(obs_names), 6), dtype=object)
for i in range(all_data.shape[0]):
    for j in range(all_data.shape[1]):
        all_data[i][j] = []

# gather the data
for directory in dirs:
    if dir_filter(directory):
        dirname = directory.name

        for i in range(len(obs_names)):
            obs_data = dp.DataSet.load(dirname, obs_names[i])
            obs_max = obs_data.transform(trans="max")
            obs_parity = obs_data.transform(trans="parity")
            all_data[i][0].append(obs_data.histogram())
            all_data[i][1].append(obs_max.histogram())
            all_data[i][2].append(obs_max.bootstrap("mean", 1000))
            all_data[i][3].append(obs_max.bootstrap("std", 1000))
            all_data[i][4].append(obs_parity.bootstrap("mean", 1000))
            all_data[i][5].append(obs_parity.bootstrap("std", 1000))
        print("And another one!")


In [ ]:


def kappa_curve(N, kappa_c, c, nu):
    return kappa_c + c * np.power(N, -nu)

# find peaks
df = dp.process_volume_kappa(all_data[10][3])
dp.write_data_frame(df, "test")
df_peak = dp.get_peak_data(df)
dp.write_data_frame(df_peak, "peak_fit")
print(df_peak)

# fit curve
popt, pcov = curve_fit(kappa_curve, df_peak["volume"], df_peak["kappa"], bounds=([-np.inf, 0, 0], [np.inf, np.inf, np.inf]))
print(popt)
plt.plot(df_peak["volume"], df_peak["kappa"])
plt.plot(df_peak["volume"], kappa_curve(df_peak["volume"], *popt))
plt.show()

In [ ]:
%matplotlib widget
for i in range(len(obs_names)):
    df_max_mean = dp.process_volume_kappa(all_data[i][2])
    df_max_sus = dp.process_volume_kappa(all_data[i][3])
    df_par_mean = dp.process_volume_kappa(all_data[i][4])
    df_par_sus = dp.process_volume_kappa(all_data[i][5])
    dp.write_data_frame(df_max_mean, f"{obs_names[i]}_max")
    dp.write_data_frame(df_max_sus, f"{obs_names[i]}_max_sus")
    dp.write_data_frame(df_par_mean, f"{obs_names[i]}_par")
    dp.write_data_frame(df_par_sus, f"{obs_names[i]}_par_sus")
    dp.volume_kappa_plot(df_max_mean, 0.5, title=f"max {obs_names[i]}")
    dp.volume_kappa_plot(df_max_sus, 0.5, title=f"max {obs_names[i]} - susceptibility")
    dp.volume_kappa_plot(df_par_mean, 0.5, title=f"par {obs_names[i]}")
    dp.volume_kappa_plot(df_par_sus, 0.5, title=f"par {obs_names[i]} - susceptibility")